# Function Dependency Equation for Free Product of 2 Cyclic Groups.

Consider $G=\langle x| x^m=1 \rangle * \langle y| y^n=1 \rangle$, where $m,n \in \mathbb{Z}_{\ge0}\setminus \{1\}$. Note that $|G|<\infty \iff mn>0$. 
Take the generating set $S=S_x\cup S_y$ for $G$, where 
$$S_x=\{x\}\cup \iota(m=0)\cdot\{x^{-1}\}, S_y=\{y\}\cup \iota(n=0)\cdot\{y^{-1}\},$$ 
and $$\iota(\mathcal{P})=\begin{cases}\{1\},&\mathcal{P}\\ \emptyset,& \lnot \mathcal{P}\end{cases}$$ with $\cdot$ denoting elementwise group multiplication on sets [$A\cdot B = \{ab: a\in A,\ b\in B\}$]. 

For $g\in G$ and $X\subseteq G$, let $F_{g,X}(t)$ be series for the set of all words in $S^*$ equivalent to $g\in G$, with proper nonempty prefixes avoiding $X$, characterized by the word length. We set up functional equations involving $F_{g,X}$, for which $\{g\}\cup X\subseteq \{v^k: k\in \mathbb{Z}\}$, and $v\in \{x,y\}$. Finally, we solve for $F(t):=F_{1,\emptyset}(t)$.

In [1]:
# choose your desired values for m,n
# for now, only work for finite G, (mn>0)
m = 2
n = 4

# Do we want word concatenation to reduce to group multiplication?
AUTO_REDUCE = True
DEFAULT_RS = None # a rewiting system in order to make group element reduction possible, will be set later


In [2]:
# need to define set-set, set-element multiplication
def set_mult(A,B):
    # turn any non-set object into a singleton set containing that object
    A,B = map(lambda S: S if S in Sets else Set([S]),[A,B])
    C= Set([a*b for a,b in cartesian_product([A,B])]) # assuming element multiplication is well defined
    if AUTO_REDUCE and DEFAULT_RS is not None:
        C = Set([DEFAULT_RS.reduce(c) for c in C])
    return C

# Let's overload the '*' operator for Set objects
# Set is a function, but returns an instance of some spectifed kind of Set class, which is accessible via .parent()
reverse_if = lambda seq, cond: list(seq) if not cond else list(reversed(seq))
Set().parent()._mul_ =  lambda self, other, switch_sides=False: set_mult(*(reverse_if([self,other], switch_sides)))

In [3]:
# compute the free product
F2.<x,y> = FreeGroup()
G = F2/ (x^m,y^n)
x,y = G.gens()

# need to write words in reduced form
Grs = G.rewriting_system()
DEFAULT_RS = Grs

In [4]:
# test it out
A = Set([x,y,G.one()])

print(x*A) # with reduction

# very simple to supress/invoke group reduction
AUTO_REDUCE = False
print(x*A) # without reduction
AUTO_REDUCE = True
print(x*A) # with reduction

{x*y, 1, x}
{x*y, x^2, x}
{x*y, 1, x}


In [5]:
# latex expression for sets
def set_to_latex(A):
    if A == Set():
        return r'\emptyset'
    return latex(A)

In [6]:
# define a space of variables indexed by the (g,X) pairs
# start with what we wish to solve, (1,\emptyset).
def series_expression(g,X):
    return 'F_{%s,%s}(t)'%(latex(g),set_to_latex(X))
var_space = {}
goal_pair = (G.one(), Set())
VAR_COUNTER = 0 # increment every time a new series variable is defined

def add_series_var(pair):
    if pair in var_space:
        return False
    global VAR_COUNTER
    g,X = pair
    var_space[pair] = var('v%d'%VAR_COUNTER, latex_name = series_expression(g,X))
    VAR_COUNTER += 1
    return True

add_series_var(goal_pair)
    

True

In [7]:
pretty_print(var_space)

{(1, {}): v0}

In [8]:
# Now get the cyclic factors

# Sage is not good with subgroups
# Gx = G.subgroup((x,))
# Gy = G.subgroup((y,))
# xx,yy = Gx.gens()+Gy.gens()

def in_cyclic_factor(el, v):
    '''
        returns if the group element is in the given cyclic factor
        el: the element in G to check
        v: either x or y, the generator of the factor 
    '''
    expr = el.syllables()
    if not expr:
        return True
    return len(expr)==1 and expr[0][0] == v
    
def subset_of_cyclic_factor(A,v):
    # assume finite sets only
    return all((in_cyclic_factor(el,v) for el in A))

In [9]:
# for finite cyclic factors, we don't want negative exponents. We redefine the __pow__ method.
if AUTO_REDUCE and DEFAULT_RS is not None:
    if '_old_pow' not in vars():
        _old_pow = type(x).__pow__
    def _new_pow(a, k):
        if k not in ZZ or k>=0:
            # case isn't broken, don't change it
            return _old_pow(a,k)

        # k is a negative integer
        ans = _old_pow(a,k)
    #     if AUTO_REDUCE:
    #         ans = DEFAULT_RS.reduce(ans)
        nans = G.one()
        for v,ep in ans.syllables():
            modval = m if v==x else n
            nans*= _old_pow(v, ep if modval==0 else ep%modval)
        return nans
    type(x).__pow__ = _new_pow

In [10]:
# For multiplication, want auto reduation to be available
# if AUTO_REDUCE and DEFAULT_RS is not None:
#     if '_old_mul' not in vars():
#         _old_mul = type(x).__mul__
#     def _new_mul(a, b):
#         return DEFAULT_RS.reduce(_old_mul(a,b))
#     type(x).__mul__ = _new_mul


In [11]:
x,x^2,x^3*x,x^-1,(x*y)^-1, x^-2,y^-3,(x*y^3)^-1, '_old_pow' in vars()

(x, x^2, x^4, x, y^3*x, 1, y, y*x, True)

In [12]:
var('t')

t

In [13]:
# construct the functional equation given the desired (g,X) pair
# assume {g}\cup X is completely in one of the cyclic factors
# CURRENTLY FINITE CASE ONLY

def get_FD_equation(pair, ret_new_pairs = True):
    '''
        returns a functional depency equation
        if ret_new_pairs is set to True, return a (equation, set of (g,X) pairs) pair instead
    '''
    if ret_new_pairs:
        newpairs = set()
    g,X = pair
    eqn = None
    one = G.one()
    oneset = Set([one])
    if one not in X:
        XU1 = X.union(oneset)
        if add_series_var((one,X)) and ret_new_pairs:
            newpairs.add((one,X))
        if add_series_var((g,XU1)) and ret_new_pairs:
            newpairs.add((g,XU1))
        if g==one:
            eqn = 1+var_space[(one,X)]*(var_space[(one,XU1)]-1)
        else:
            eqn = var_space[(one,X)]*var_space[(g,XU1)]
    elif subset_of_cyclic_factor(X,x):
        if g!=one:
            eqn = t*ZZ(g==x)
            if x not in X:
                [g] = x^-1*Set([g]) # use set so auto reduction is implemented
                X = x^-1*X
                if add_series_var((g,X)) and ret_new_pairs:
                    newpairs.add((g,X))
                eqn += t*var_space[(g,X)]
        else:
            set_yinv = Set([y^-1]) 
            if add_series_var((y^-1,set_yinv)) and ret_new_pairs:
                newpairs.add((y^-1,set_yinv))
            eqn = 1+t*var_space[(y^-1,set_yinv)]
            if x not in X:
                g = x^-1
                X = x^-1*X
                if add_series_var((g,X)) and ret_new_pairs:
                    newpairs.add((g,X))
                eqn += t*var_space[(g,X)]
    else:
        # the y factor case
        if g!=one:
            eqn = t*ZZ(g==y)
            if y not in X:
                [g] = y^-1*Set([g])
                X = y^-1*X
                if add_series_var((g,X)) and ret_new_pairs:
                    newpairs.add((g,X))
                eqn += t*var_space[(g,X)]
        else:
            set_xinv = Set([x^-1]) 
            if add_series_var((x^-1,set_xinv)) and ret_new_pairs:
                newpairs.add((x^-1,set_xinv))
            eqn = 1+t*var_space[(x^-1,set_xinv)]
            if y not in X:
                g = y^-1
                X = y^-1*X
                if add_series_var((g,X)) and ret_new_pairs:
                    newpairs.add((g,X))
                eqn += t*var_space[(g,X)]
    eqn = var_space[pair]==eqn
    return (eqn,newpairs) if ret_new_pairs else eqn
    


In [14]:
# Let us build the system of equations
system = []
queue = [goal_pair]

# use BFS
while queue:
    pair = queue.pop(0)
    eqn, newp = get_FD_equation(pair)
    queue.extend(newp)
    system.append(eqn)


In [15]:
# Here are the equations
for eqn in system:
    print(latex(eqn),r'\\')
print()
print()
for eqn in system:
    #print(eqn)
    pretty_print(eqn)

{F_{1,\emptyset}(t)} = {F_{1,\emptyset}(t)} {\left({F_{1,\left\{1\right\}}(t)} - 1\right)} + 1 \\
{F_{1,\left\{1\right\}}(t)} = t {F_{y^{3},\left\{y^{3}\right\}}(t)} + t {F_{x,\left\{x\right\}}(t)} + 1 \\
{F_{y^{3},\left\{y^{3}\right\}}(t)} = {F_{1,\left\{y^{3}\right\}}(t)} {F_{y^{3},\left\{y^{3}, 1\right\}}(t)} \\
{F_{x,\left\{x\right\}}(t)} = {F_{1,\left\{x\right\}}(t)} {F_{x,\left\{1, x\right\}}(t)} \\
{F_{1,\left\{y^{3}\right\}}(t)} = {F_{1,\left\{y^{3}\right\}}(t)} {\left({F_{1,\left\{y^{3}, 1\right\}}(t)} - 1\right)} + 1 \\
{F_{y^{3},\left\{y^{3}, 1\right\}}(t)} = t {F_{y^{2},\left\{y^{3}, y^{2}\right\}}(t)} \\
{F_{1,\left\{x\right\}}(t)} = {\left({F_{1,\left\{1, x\right\}}(t)} - 1\right)} {F_{1,\left\{x\right\}}(t)} + 1 \\
{F_{x,\left\{1, x\right\}}(t)} = t \\
{F_{1,\left\{y^{3}, 1\right\}}(t)} = t {F_{y^{3},\left\{y^{3}, y^{2}\right\}}(t)} + t {F_{x,\left\{x\right\}}(t)} + 1 \\
{F_{y^{2},\left\{y^{3}, y^{2}\right\}}(t)} = {F_{1,\left\{y^{3}, y^{2}\right\}}(t)} {F_{y^{2},\left\{

v0 == v0*(v1 - 1) + 1

v1 == t*v2 + t*v3 + 1

v2 == v4*v5

v3 == v6*v7

...

v19 == t

v20 == v18*v23

v21 == 0

v22 == t*v3 + 1

v23 == 0

In [17]:
# Finally solve the system

# reports an empty list

# save so we dont lose it
orig_system = system

In [18]:
# first, isolate variables on the LHS

for i, eqn in enumerate(system):
    system[i] = solve(eqn,eqn.lhs())[0]
    

In [19]:
for eqn in system:
    pretty_print(eqn)

v0 == -1/(v1 - 2)

v1 == t*v2 + t*v3 + 1

v2 == v4*v5

v3 == v6*v7

v4 == -1/(v8 - 2)

...

v19 == t

v20 == v18*v23

v21 == 0

v22 == t*v3 + 1

v23 == 0

In [20]:
# now make some substitutions

change = 1
numit = 0

while change:
    change = 0
    # build a dependency graph
    eq_by_lhs = {eq.lhs():eq for eq in system}
    dep_gr = {eq.lhs():set(eq.rhs().args())-{t} for eq in system}
    vars_det = [v for v,args in dep_gr.items() if not args] # the completely solved vars

    rev_dep_gr = {v:set() for v in eq_by_lhs} # diagraph by reversing the arcs of dep_gr
    for v,ag in dep_gr.items():
        for w in ag:
            rev_dep_gr[w].add(v)

    # apply BFS to make the substitutions possible
    vis = set(vars_det)
    queue = list(vis)
    done = set()
    while queue:
        nxtvar = queue.pop(0)
        # check if all RHS variables are visited before
        sub_dict = {}
        for dps in dep_gr[nxtvar]:
            if dps not in done:
                continue # can't substitute, a RHS varible is not yet reduced
            sub_dict[dps] = eq_by_lhs[dps].rhs()
            change = 1
        eq_by_lhs[nxtvar] = eq_by_lhs[nxtvar].subs(sub_dict)
        eq_by_lhs[nxtvar].expand()
        for pt in rev_dep_gr[nxtvar]:
            if pt not in vis:
                queue.append(pt)
                vis.add(pt)
        done.add(nxtvar)
    
    for i, eq in enumerate(system):
        system[i] = eq_by_lhs[eq.lhs()]
    numit+=1

print('Number of iteration taken to substitute',numit)
        


Number of iteration taken to substitute 3


In [21]:
for eq in eq_by_lhs.values():
    pretty_print(eq)

v0 == (1/(t^4*v12*v18/(t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) - 1) - t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) + 1))

v1 == -t^4*v12*v18/(t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) - 1) + t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) + 1

v2 == -t^3*v12*v18/(t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) - 1)

v3 == t/(t^4*v12*v18/(t^2*v6 - 1) + 1)

v4 == -1/(t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) - 1)

...

v19 == t

v20 == 0

v21 == 0

v22 == t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) + 1

v23 == 0

In [22]:
# get rid of every irrelevant variable
start = var_space[goal_pair]
eq_by_lhs = {eq.lhs():eq for eq in system}
dep_gr = {eq.lhs():set(eq.rhs().args())-{t} for eq in system}

comp_set = {start}
comp = [start]
ind = 0
while ind<len(comp):
    for v in dep_gr[comp[ind]]:
        if v not in comp_set:
            comp.append(v)
            comp_set.add(v)
    ind += 1

system = [eq for eq in system if eq.lhs() in comp_set]

In [23]:
pretty_print(comp)
for eq in system:
    print(eq)
    pretty_print(eq)

[v0, v6, v12, v18]

v0 == (1/(t^4*v12*v18/(t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) - 1) - t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) + 1))


v0 == (1/(t^4*v12*v18/(t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) - 1) - t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) + 1))

v6 == (1/(t^4*v12*v18/(t^2*v6 - 1) + 1))


v6 == (1/(t^4*v12*v18/(t^2*v6 - 1) + 1))

v12 == -1/(t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) - 1)


v12 == -1/(t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) - 1)

v18 == -1/(t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) - 1)


v18 == -1/(t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) - 1)

In [ ]:
# make every equation eplicit. delete loops
# maybe not a good idea due to the posssible interference of radicals

# for i, eqn in enumerate(system):
#     system[i] = solve(eqn,eqn.lhs())[0]

In [24]:
for eq in system:
    print(eq)
    pretty_print(eq)

v0 == (1/(t^4*v12*v18/(t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) - 1) - t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) + 1))


v0 == (1/(t^4*v12*v18/(t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) - 1) - t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) + 1))

v6 == (1/(t^4*v12*v18/(t^2*v6 - 1) + 1))


v6 == (1/(t^4*v12*v18/(t^2*v6 - 1) + 1))

v12 == -1/(t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) - 1)


v12 == -1/(t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) - 1)

v18 == -1/(t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) - 1)


v18 == -1/(t^2/(t^4*v12*v18/(t^2*v6 - 1) + 1) - 1)

In [25]:
# order the component based on the shortest distance from the start in decreasing order
# BFS already put them in increasing order
comp.reverse()
comp.pop(); # get rid of start; the only variable we don't want to eliminate
comp

[v18, v12, v6]

In [26]:
def eliminate(system,xs, show_steps = False, fully_simplify=False, quit_at_zero= False):
    if show_steps:
        sols = []
        for x in xs:
            sols.append((x, eliminate(system,[x], fully_simplify=fully_simplify)))
            system = sols[-1][1]
            if quit_at_zero and system == [0]*len(system):
                return sols
        return sols
    sols = maxima.eliminate(system,xs)
    func = lambda ex: ex.simplify_full() if fully_simplify else ex
    return [func(expr.sage()) for expr in sols]

In [27]:
# Expression involving t and F(t) to set = 0

impl_solset = []
steps = eliminate(system, comp, True, quit_at_zero=True)
if steps and steps[-1][-1]==[0]*len(steps[-1][-1]):
    steps.pop()
if steps:
    impl_solset = steps[-1][-1]


'''
possible_impl_solns = []
for comp_perm in Permutations(comp):
    comp_perm = list(comp_perm)
    print(comp_perm,type(comp_perm))
    possible_impl_solns_perm = eliminate(system, comp_perm)
    possible_impl_solns.extend(possible_impl_solns_perm)
    for impl_soln in possible_impl_solns_perm:
        pretty_print(impl_soln)
        if impl_soln!=0:
            break # found an actual solution
    else:
        continue
    break
''';

In [28]:
print(len(impl_solset))
for sol in impl_solset:
    pretty_print(sol)
    print(sol.args())

    
#solve(system, comp, algorithm='sympy')
#impl_soln(v0=sqrt(1/(1-4*t^2))).simplify_full()

3


-(t^12*v0^2*v12^3 + (t^14*v0^2*v12 + (t^16 - 3*t^14 + 3*t^12 - t^10)*v0^2 + (t^14 - 2*t^12 + t^10)*v0)*v6^5 - (3*t^10 + t^8)*v0^2*v12^2 - (5*(t^14 - 3*t^12 + 3*t^10 - t^8)*v0^2 + 5*(t^12 - 2*t^10 + t^8)*v0 - (2*t^14*v0 + (3*t^16 - 3*t^14 - 5*t^12)*v0^2)*v12)*v6^4 + (10*(t^12 - 3*t^10 + 3*t^8 - t^6)*v0^2 + (3*t^16*v0^2 + t^14*v0)*v12^2 + 10*(t^10 - 2*t^8 + t^6)*v0 + (t^10 - 2*(6*t^14 - 6*t^12 - 5*t^10)*v0^2 - (7*t^12 + t^10)*v0)*v12)*v6^3 - (t^6 - 3*t^4 + 3*t^2 - 1)*v0^2 + (t^16*v0^2*v12^3 - 10*(t^10 - 3*t^8 + 3*t^6 - t^4)*v0^2 - (2*t^12*v0 + (9*t^14 + t^12)*v0^2)*v12^2 - 10*(t^8 - 2*t^6 + t^4)*v0 - (3*t^8 - 2*(9*t^12 - 9*t^10 - 5*t^8)*v0^2 - 3*(3*t^10 + t^8)*v0)*v12)*v6^2 - (t^4 - 2*t^2 + 1)*v0 - (t^4 - (3*t^8 - 3*t^6 - t^4)*v0^2 - (t^6 + t^4)*v0)*v12 - (2*t^14*v0^2*v12^3 - 5*(t^8 - 3*t^6 + 3*t^4 - t^2)*v0^2 - (t^10*v0 + (9*t^12 + 2*t^10)*v0^2)*v12^2 - 5*(t^6 - 2*t^4 + t^2)*v0 - (3*t^6 - (12*t^10 - 12*t^8 - 5*t^6)*v0^2 - (5*t^8 + 3*t^6)*v0)*v12)*v6)*t^16*v12^3

(t, v0, v12, v6)


((t^2 - 1)*v0*v12^3 - (t^6*v0*v12^2 + (t^8 - t^6)*v0*v12^3)*v6^3 + (v0 - 1)*v12^2 - (t^4*v0 - 3*(t^6 - t^4)*v0*v12^3 - (3*t^4*v0 - t^4)*v12^2 - (t^4*v0 + t^4)*v12)*v6^2 + (v0 + 1)*v12 - (3*(t^4 - t^2)*v0*v12^3 - 2*t^2*v0 + (3*t^2*v0 - 2*t^2)*v12^2 + 2*(t^2*v0 + t^2)*v12)*v6 - v0)*t^16*v12^3

(t, v0, v12, v6)


(t^8*v0*v6^5 - (3*t^6*v0 - t^6)*v6^4 - (3*t^4 + (t^6 - 3*t^4)*v0)*v6^3 + (3*t^2 + (3*t^4 - t^2)*v0)*v6^2 - (3*t^2*v0 + 1)*v6 + v0)*t^12*v12^3

(t, v0, v12, v6)


In [29]:
# Solve if we can
var_rem = comp[-len(impl_solset)+1:]+[start]
print(var_rem)

have_expl = 1
try:
    expl_slns = solve(impl_solset, var_rem, solution_dict=True) # can we get explicit solutions?
    print(expl_slns)
except: 
    # can't get explicit solutions. try for implicit ones. perhaps reduced a bit?
    impl_solset = solve(impl_solset, var_rem)
    have_expl = 0
    for sol in impl_solset:
        pretty_print(sol)
    

[v12, v6, v0]


((t^2 - 1)*v0*v12^3 - (t^6*v0*v12^2 + (t^8 - t^6)*v0*v12^3)*v6^3 + (v0 - 1)*v12^2 - (t^4*v0 - 3*(t^6 - t^4)*v0*v12^3 - (3*t^4*v0 - t^4)*v12^2 - (t^4*v0 + t^4)*v12)*v6^2 + (v0 + 1)*v12 - (3*(t^4 - t^2)*v0*v12^3 - 2*t^2*v0 + (3*t^2*v0 - 2*t^2)*v12^2 + 2*(t^2*v0 + t^2)*v12)*v6 - v0)*t^16*v12^3

(t^8*v0*v6^5 - (3*t^6*v0 - t^6)*v6^4 - (3*t^4 + (t^6 - 3*t^4)*v0)*v6^3 + (3*t^2 + (3*t^4 - t^2)*v0)*v6^2 - (3*t^2*v0 + 1)*v6 + v0)*t^12*v12^3

-(t^12*v0^2*v12^3 + (t^14*v0^2*v12 + (t^16 - 3*t^14 + 3*t^12 - t^10)*v0^2 + (t^14 - 2*t^12 + t^10)*v0)*v6^5 - (3*t^10 + t^8)*v0^2*v12^2 - (5*(t^14 - 3*t^12 + 3*t^10 - t^8)*v0^2 + 5*(t^12 - 2*t^10 + t^8)*v0 - (2*t^14*v0 + (3*t^16 - 3*t^14 - 5*t^12)*v0^2)*v12)*v6^4 + (10*(t^12 - 3*t^10 + 3*t^8 - t^6)*v0^2 + (3*t^16*v0^2 + t^14*v0)*v12^2 + 10*(t^10 - 2*t^8 + t^6)*v0 + (t^10 - 2*(6*t^14 - 6*t^12 - 5*t^10)*v0^2 - (7*t^12 + t^10)*v0)*v12)*v6^3 - (t^6 - 3*t^4 + 3*t^2 - 1)*v0^2 + (t^16*v0^2*v12^3 - 10*(t^10 - 3*t^8 + 3*t^6 - t^4)*v0^2 - (2*t^12*v0 + (9*t^14 + t^12)*v0^2)*v12^2 - 10*(t^8 - 2*t^6 + t^4)*v0 - (3*t^8 - 2*(9*t^12 - 9*t^10 - 5*t^8)*v0^2 - 3*(3*t^10 + t^8)*v0)*v12)*v6^2 - (t^4 - 2*t^2 + 1)*v0 - (t^4 - (3*t^8 - 3*t^6 - t^4)*v0^2 - (t^6 + t^4)*v0)*v12 - (2*t^14*v0^2*v12^3 - 5*(t^8 - 3*t^6 + 3*t^4 - t^2)*v0^2 - (t^10*v0 + (9*t^12 + 2*t^10)*v0^2)*v12^2 - 5*(t^6 - 2*t^4 + t^2)*v0 - (3*t^6 - (12*t^10 - 12*t^8 - 5*t^6)*v0^2 - (5*t^8 + 3*t^6)*v0)*v12)*v6)*t^16*v12^3

In [30]:
if have_expl:
    for sol in expl_slns:
        F = sol[start]
        L = limit(F,t=0)
        pretty_print(LatexExpr(r'\lim_{t\to 0}%s=%s'%(latex(start),L)))
        if L!=1:
            print('limit at t=0 is different from 1, discard!')
        else:
            print('possible desired series solutions!')
            #pretty_print(F)
            prec = 70
            pretty_print('Series Expansion around %s of degree up to %d: '%(LatexExpr('t=0'),prec))
            pretty_print(F.taylor(t, 0, prec))

# Solving a system iteratively

Suppose we are given a system of the form
$$\begin{align*}
\Phi_1 (v_1,...v_q) &= 0\\
\Phi_2 (v_1,...v_q) &= 0\\
\vdots\\
\Phi_r (v_1,...v_q) &= 0
\end{align*}$$ where $q\ge r$. We want to solve for $v_1,...,v_r$ in terms of the remaining $q-r$ variables.
We can do this approximately using a fixed point iterative method, assumming convergence.

First, rewrite the $i$-th equation above as $$\Phi_i (v_1,...,v_q)+v_i = v_i.$$
Generate iterates $v_i^{(k)}$ with $v_i^{(0)}$ determined, with the update formula as $$\Phi_i (v_1^{(k)},...,v_q^{(k)})+v_i^{(k)} = v_i^{(k+1)}.$$ For large $k$, the iterates make an approximate solution.

In the case where $r=q-1$ and $v_q=t$, and the system is algebraic, we can expect approximations to the power series solution of $v_i$ to be increasingly accurate. If we take $v_i^{(0)}=0$ for each $i$, then we should get increasing higher oder terms each iteration, with $v_i^{(k)}$ converging to the series as $k\to \infty$.

In [31]:
def homogenize(eqn):
    '''
    eqn: an equation or expression (to set = 0)
    returns an expression, to set = 0, that is equivalent to applying eqn
    '''
    if eqn.is_relational():
        return eqn.rhs()-eqn.lhs()
    return eqn


def iter_solve(system, vrs, vinit=None, maxiter=5, fully_simplify=False):
    if len(system)<len(vrs):
        raise Exception('number of variables must be at least the number of equations')
    if not system:
        return [] # no system to solve
    if vinit is None:
        vinit = {v:0*t for v in vrs}
    system = list(map(homogenize, system))
    it_sols = [vinit]
    k = 1
    while k<=maxiter:
        vrs_k = {}
        for vi,eqi in zip(vrs,system):
            vrs_k[vi] = it_sols[-1][vi]+eqi.subs(it_sols[-1])
            if fully_simplify:
                vrs_k[vi] = vrs_k[vi].simplify_full()
        it_sols.append(vrs_k)
        k+=1
    return it_sols
    

In [35]:
for ans in iter_solve(orig_system,list(var_space.values()),fully_simplify=False,maxiter=10):
    pretty_print(ans[start].taylor(t,0,5))

0

t^4 + t^2 + 1

t^4 + t^2 + 1

t^2 + 1

-2*t^4 + t^2 + 1

-6*t^4 + t^2 + 1

-14*t^4 + t^2 + 1

-30*t^4 + t^2 + 1

-62*t^4 + t^2 + 1

TypeError: ECL says: Console interrupt.

In [ ]:
print(ss,file=open('ss.txt','w'))

In [ ]:
eg = [-((t^3*v0^2 - v0^2 - 2*v0 - 1)*v13^3 + (3*v0^2 + 4*v0 + 1)*v13^2 + v0^2 - (3*v0^2 + 2*v0)*v13)*t^24*v13^6, (t^6*v13^5 - t^6*v13^4 - 2*t^3*v13^3 + 2*t^3*v13^2 - (t^3 - 1)*v13 - 1)*t^18*v13^4]
maxima.eliminate(eg, [v13])

In [ ]:
for v,ss in eliminate(system, [v3,v15,v2,v4,v13], True):
    print('+'*100)
    pretty_print(v,':')
    for s in ss:
        print('|'*50)
        print(bool(s==0))
        pretty_print(s)
    print (ss)
    if len(ss)==1:
        pretty_print(ss[0])

In [ ]:
solve(impl_solset,[v0,v13])

In [ ]:
type(expl_slns[0][v0])

v0